In [1]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('cmudict')
nltk.download('brown')
nltk.download('stopwords')

#need tagger to extract data of no themes
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
import pandas as pd
import numpy as np
import csv
import re
from nltk import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import cmudict
from nltk.corpus import brown
from nltk.corpus import stopwords
from collections import defaultdict
from collections import Counter

from google.colab import drive 
drive.mount('/content/gdrive')

data = pd.read_csv('gdrive/My Drive/ProfoundPoet/PoetryFoundationData.csv')

Mounted at /content/gdrive


In [3]:
poems = data['Poem']
themes = data['Tags']
titles = data['Title']

## 1. We  will need trigrams (sequential data) of all poems and brown corpus words in order to generate our poems

### Brown Trigrams:
We need to extract trigrams with the sentence boundaries

In [4]:
brown_trigrams = [tri for sent in brown.sents() for tri in nltk.ngrams(sent, 3)]
brown_cfd = nltk.ConditionalFreqDist(((tri[0].lower(),tri[1].lower()), tri[-1].lower()) for tri in brown_trigrams)

###Poem Trigrams:

This will take us some time, as we want to extract the trigrams in units of 'line' but our poems need to be tokenized into lines and tokens

In [5]:
# From the test case above, created a function to tokenize all poems in the dataset
def tokenize_poem_by_line(poem):
  # want to maintain the line splitting/division within poems
  poem_split = re.split(r'[\n\r]+| + ', poem)
  # want to only work with the words in the poem
  return [re.findall(r'[a-zA-Z0-9\']+', line) for line in poem_split if line!='' and line!=' ']

In [6]:
#tokenizing all poems
tokenized_poems = [tokenize_poem_by_line(poem) for poem in poems]

In [7]:
poem_trigrams = [tri for poem in tokenized_poems for line in poem for tri in nltk.ngrams(line, 3)]
poem_cfd = nltk.ConditionalFreqDist(((tri[0].lower(),tri[1].lower()), tri[-1].lower()) for tri in poem_trigrams)

###Check formation

Let's check if your trigrams have been created properly

In [8]:
print("Example entry in brown:", list(brown_cfd.items())[0], ", Length of brown_cfd:", len(brown_cfd.conditions()))
print("Example entry in poem:",list(poem_cfd.items())[0], ", Length of poem_cfd:", len(poem_cfd.conditions()))

Example entry in brown: (('the', 'fulton'), FreqDist({'county': 3, "ordinary's": 1, 'tax': 1, 'health': 1})) , Length of brown_cfd: 414945
Example entry in poem: (('dog', 'bone'), FreqDist({'stapler': 1})) , Length of poem_cfd: 993376


##Getting pair of tokens that come at the end of a poem's line (reference for end-point)

In [9]:
def get_end_tokens():
  return nltk.FreqDist((line[-2],line[-1]) for poem in tokenized_poems for line in poem if len(line)>2)

In [10]:
line_endings = get_end_tokens()

common_line_endings = [first for first, second in line_endings.most_common(3000)]

In [11]:
common_line_endings[:5]

[('the', 'world'),
 ('to', 'me'),
 ('the', 'sun'),
 ('the', 'sky'),
 ('the', 'sea')]

##Grouping / Identifying frequently appearing words by theme

###Assigning poems with no tag with tags inferred from their titles

In [12]:
stopwords = nltk.corpus.stopwords.words()

unique_themes = set(taglist for taglist in themes if isinstance(taglist,str))
unique_themes = set(theme for taglist in unique_themes for theme in re.split(r',\s*|\s*&\s*|\s+',taglist) if theme and theme.lower() not in stopwords)

In [13]:
len(unique_themes)

180

In [14]:
lemmatizer = nltk.WordNetLemmatizer()

# Look for synset names, which are a generalization or representation of words, in the hypernym path to see if there are any overlaps with the unique themes

#Function that extracts the synset names of the hypernyms
def hypernym_names(synsets):
  return [word for synset in synsets for syn in synset.hypernym_paths()[0] for word in re.split(r'_', syn.name()[:-5])]

def no_tag(title):
  #Extract the words from the title
  keywords = re.findall(r'[a-zA-Z]+',title)
  #There are many synsets given one word, so to extract relevant synsets (as much as possible) we only gather the synsets of the same pos type
  keyword_pos_tags = nltk.pos_tag(keywords)

  #we collect keywords as they are if they are in the form of a noun, verb, or adjective in case there are no overlaps with the unique themes
  keywords = set()
  synsets = []
  for keyword, pos in keyword_pos_tags:
    if pos[0] == 'N':
      keywords.add(keyword)
      synsets += wn.synsets(keyword, pos = wn.NOUN) 
    elif pos[0] == 'V':
      keywords.add(keyword)
      synsets += wn.synsets(keyword, pos = wn.VERB)
    elif pos[0] == 'J':
      keywords.add(keyword)
      synsets += wn.synsets(keyword, pos = wn.ADJ)
    else:
      continue
  
  names = hypernym_names(synsets)
  manual = set()

  for name in names:
    for theme in unique_themes:
      lemma = lemmatizer.lemmatize(theme.lower())
      if name.lower() == lemma: manual.add(theme.lower())
  
  if not manual:
    return list(keywords)

  return list(manual)

### Tokenizing themes in the poem

In [15]:
#Here, I extracted the tags and tokenized them and added the list of tags per poem to the list 'themes'
poem_themes = []
for i, tag_list in enumerate(themes):
  if isinstance(tag_list, str):
    poem_themes.append(re.split(r',\s*|\s*&\s*',tag_list))
  else:
    poem_themes.append(no_tag(titles[i]))

###Finding the frequently appearing words

We only want to inspect words that are nouns, verbs, or adjectives so we use the nltk pos_tagger.

In [16]:
def unique_words_in_poem(poem):
  words = [word.lower() for word in re.findall(r'[a-zA-Z\']+', poem) if word not in stopwords]
  pos_words =  nltk.pos_tag(words)
  return nltk.FreqDist(word.lower() for word,tag in pos_words if tag[0] in ["N","V","J"])

In [17]:
unique_words_poem = [unique_words_in_poem(poem) for poem in poems]

###Associating the Frequency Distributions of each poem with their theme tags

First, associating the unique words with all themes the poem is associated with

In [18]:
unique_words_dataset = list(zip(unique_words_poem, poem_themes))

Next, creating a dictionary that has key of an individual theme, and value of a frequency distribution of unique words.
We will use a lemmatizer for the keys for consistency

In [19]:
group_words_by_theme = defaultdict(nltk.FreqDist)

for (fdist, themes) in unique_words_dataset:
  for theme in themes:
    if theme:
      key = lemmatizer.lemmatize(theme.lower())
      group_words_by_theme[key].update(fdist)

Let's check if the dictionary was well-created

In [20]:
group_words_by_theme['class']

FreqDist({'i': 1857, 'day': 177, 'white': 166, 'time': 147, 'black': 135, 'world': 134, 'night': 129, "an'": 126, 'light': 120, 'eyes': 118, ...})

In [233]:
theme_list = [key for key in group_words_by_theme.keys() if group_words_by_theme[key].N() > 180000]
theme_list

['science',
 'time',
 'religion',
 'activity',
 'history',
 'living',
 'relationship',
 'nature',
 'social commentaries',
 'politics',
 'death',
 'love',
 'art']

Let's extract the frequent words that occur more than COMMON_COUNT times in the theme. The reason why we set a threshold is because we want to remove words that only appear in certain poems (like proper nouns)

eg) Theme : Love

I rode a ferrari to get to her -> 'horse' is not relevant to the theme 'love'

In [21]:
COMMON_COUNT = 20

def get_frequent_words(theme, min_count = COMMON_COUNT):
  return [word for word,count in group_words_by_theme[theme].most_common() if count > min_count]

Let's check if the common_words are well selected and well reflect the theme

In [22]:
common = get_frequent_words("love")

print("Number of words that occure more than", COMMON_COUNT, "times:", len(common))
print("Top 20 most occuring words:", common[:20])

Number of words that occure more than 20 times: 1805
Top 20 most occuring words: ['i', 'love', 'heart', 'night', 'eyes', 'time', 'light', 'thy', 'day', 'life', 'thou', 'hand', 'make', 'body', 'world', 'sweet', 'made', 'white', 'thee', 'things']


##Looking for words in freq_words that start with the alliteration letter

In [23]:
def alliteration_freq_words(letter, frequent_words):
  return [word for word in frequent_words if word[0].startswith(letter)]

Let's check if the alliteration are well selected

In [24]:
alliteration_words = alliteration_freq_words("a", common)

print("Number of frequent words that start with the letter, 'a': ", len(alliteration_words))
print("Top 20 most occuring words:", common[:20])

Number of frequent words that start with the letter, 'a':  51
Top 20 most occuring words: ['i', 'love', 'heart', 'night', 'eyes', 'time', 'light', 'thy', 'day', 'life', 'thou', 'hand', 'make', 'body', 'world', 'sweet', 'made', 'white', 'thee', 'things']


##Looking for words with a certain rhythm

In [25]:
cmu = cmudict.dict()
pron_dict = dict((key, ''.join(value)) for (key,values) in cmu.items() for value in values)

In [26]:
def iambic_stress_match(pron):
  if [char for char in pron if char.isdigit()] == ['0', '1']: 
    return True
  return False

def trochaic_stress_match(pron):
  if [char for char in pron if char.isdigit()] == ['1', '0']: 
    return True
  return False

def dactylic_stress_match(pron):
  if [char for char in pron if char.isdigit()] == ['1', '0', '0']: 
    return True
  return False
  
def anapestic_stress_match(pron):
  if [char for char in pron if char.isdigit()] == ['0', '0', '1']: 
    return True
  return False

In [27]:
def rhythm_word_candidates(func, common):
  collection = []
  for word in cmu.keys():
    if func(pron_dict[word]) and word in common:
      collection.append(word)
  return collection

Let's check if the alliteration are well selected

In [28]:
rhythm_words = rhythm_word_candidates(dactylic_stress_match, common)

print("Number of frequent words with a dactylic sound", len(rhythm_words))
print("Top 20 most occuring words:", rhythm_words[:20])

Number of frequent words with a dactylic sound 36
Top 20 most occuring words: ['animal', 'animals', 'beautiful', 'carrying', 'century', 'company', 'curious', 'delicate', 'difficult', 'ecstasy', 'gathering', 'glorious', 'happiness', 'images', 'infinite', 'innocence', 'liberty', 'loneliness', 'memories', 'memory']


##Synonyms of the theme using Wordnet lexicon

In [29]:
def get_theme_synonyms(theme):
  synsets = wn.synsets(theme)
  synonyms = set(re.sub(r'_', ' ', lemma) for syn in synsets for lemma in syn.lemma_names())
  return synonyms

Let's check if the synonyms are well selected

In [30]:
theme_synonyms = get_theme_synonyms('love')

print("Number of theme synonyms", len(theme_synonyms))
print("Display some theme synonyms:", list(theme_synonyms)[:20])

Number of theme synonyms 36
Display some theme synonyms: ['have sex', 'honey', 'make love', 'beloved', 'hump', 'jazz', 'sexual love', 'dear', 'do it', 'enjoy', 'erotic love', 'have it away', 'fuck', 'love life', 'bed', 'making love', 'lovemaking', 'sleep together', 'love', 'have intercourse']


##Check if the group of key words identified has any overlaps

1. alliteration & rhythm

In [31]:
aliteration_rhythm_intersect = set(alliteration_words).intersection(set(rhythm_words))
aliteration_rhythm_intersect

{'animal', 'animals'}

2. alliteration & synonyms

In [32]:
aliteration_synonyms_intersect = set(alliteration_words).intersection(set(theme_synonyms))
aliteration_synonyms_intersect

set()

3. rhythm & synonyms

In [33]:
rhythm_synonyms_intersect = set(rhythm_words).intersection(set(theme_synonyms))
rhythm_synonyms_intersect.union(set(word for word in rhythm_words for synonym in theme_synonyms if word in synonym))
rhythm_synonyms_intersect

set()

4. alliteration & rhythm & synonyms

In [34]:
alliteration_rhythm_synonyms_intersect = (set(rhythm_words).intersection(set(theme_synonyms))).intersection(set(alliteration_words))
alliteration_rhythm_synonyms_intersect

set()

###Our hierarchy of search-order is 4>(1,2,3)

##Generation

In [285]:
print("These are the themes", theme_list)
theme = input("Theme of poem:")
print("You have selected theme:", theme)

theme_synonyms = get_theme_synonyms(theme)

print("These are the possible topics you can start your poem with:")
print()

for (i, word) in enumerate(theme_synonyms):
  print("{:<20}".format(word), end= "")
  if i%3 == 0 and i!=1 : print()

These are the themes ['science', 'time', 'religion', 'activity', 'history', 'living', 'relationship', 'nature', 'social commentaries', 'politics', 'death', 'love', 'art']
Theme of poem:politics
You have selected theme: politics
These are the possible topics you can start your poem with:

political sympathies
politics            political relation  political science   
government          

In [286]:
while True:
  topic = input("If you don't want to specify a topic, press enter. Otherwise, type the topic you want:")
  if topic not in theme_synonyms and topic != '':
    print("Sorry, it is not a valid topic. Please try again.")
  else:
    break
print()

If you don't want to specify a topic, press enter. Otherwise, type the topic you want:government



In [287]:
topic_synonyms = [syn for top in topic.split() for syn in get_theme_synonyms(top)]

synonyms = theme_synonyms.union(topic_synonyms)

freq_words_for_theme = get_frequent_words(theme.lower(), 50)

In [288]:
len(freq_words_for_theme)

783

In [289]:
alliteration = input("Select the letter for alliteration:")


print("Your alliteration letter will be:", alliteration)

alliteration_words = alliteration_freq_words(alliteration, freq_words_for_theme)

if alliteration == '' : alliteration_words = []
print(len(alliteration_words))

Select the letter for alliteration:
Your alliteration letter will be: 
0


In [290]:
rhythm_functions = [iambic_stress_match, trochaic_stress_match, dactylic_stress_match, anapestic_stress_match]
rhythm_candidates = []
for func in rhythm_functions:
  rhythm_candidates.append(rhythm_word_candidates(func, freq_words_for_theme))

rhythm, rhythm_words = max([rhythm for rhythm in enumerate(rhythm_candidates)], key = lambda x: len(x[1]))

print(rhythm_functions[rhythm], len(rhythm_words))

rhytm_func = rhythm_functions[rhythm]

<function trochaic_stress_match at 0x7fec86b075b0> 197


Checking results

In [291]:
print("Portion of alliteration words:")
for (i, word) in enumerate(alliteration_words[:9]):
  print("{:<20}".format(word), end= "")
  if (i+1)%3 == 0 : print()
print()

print("Portion of rhythm words:")
for (i, word) in enumerate(rhythm_words[:9]):
  print("{:<20}".format(word), end= "")
  if (i+1)%3 == 0 : print()
print()

print("Portion of synonym words:")
for (i, word) in enumerate(list(synonyms)[:9]):
  print("{:<20}".format(word), end= "")
  if (i+1)%3 == 0 : print()
print()

print("Portion of frequent words:")
for (i, word) in enumerate(freq_words_for_theme[:9]):
  print("{:<20}".format(word), end= "")
  if (i+1)%3 == 0 : print()
print()

Portion of alliteration words:

Portion of rhythm words:
ancient             angels              answer              
army                baby                battle              
beauty              bloody              bodies              

Portion of synonym words:
political sympathiespolitics            political relation  
political science   authorities         administration      
government activity regime              governing           

Portion of frequent words:
i                   time                white               
night               world               day                 
life                light               black               



In [292]:
brown_cfd_sum_avg = sum(brown_cfd[cond].N() for cond in brown_cfd.conditions()) / len(brown_cfd.conditions())
poem_cfd_sum_avg = sum(poem_cfd[cond].N() for cond in poem_cfd.conditions()) / len(poem_cfd.conditions())

def calc_continuity(next_pair, brown):
  if brown:
    proportion = (float(brown_cfd[next_pair].N()) / brown_cfd_sum_avg) * 10000
    if proportion> 1: return 1
    else: return proportion 
  else:
    proportion = (float(poem_cfd[next_pair].N()) / poem_cfd_sum_avg) * 20000
    if proportion> 1: return 1
    else: return proportion

In [293]:
def generate_word(pair, sent):
  word_rank = defaultdict(float)

  for word in brown_cfd[pair].keys():
    if word.isalpha(): 
      brown_total =  (0.4 + brown_cfd[pair].freq(word) + calc_continuity((pair[-1], word), True))
      word_rank[(pair,word)] += brown_total
    else: continue

  for word in poem_cfd[pair].keys(): 
    if word.isalpha():
      poem_total = (0.45 + poem_cfd[pair].freq(word) + calc_continuity((pair[-1], word), False))
      word_rank[(pair,word)] += poem_total
    else: continue

  all_pairs = set(brown_cfd.conditions()).union(set((poem_cfd.conditions())))
  first_word = set(w1 for (w1,w2) in all_pairs)
  
  for key in word_rank.keys():
    pair, word = key
    #not in frequent words but satisfies alliteration and rhythm function
    if word.startswith(alliteration): word_rank[key] += 0.2

    if rhytm_func(word): word_rank[(pair,word)] += 0.1

    if word in alliteration_words:
      word_rank[key] += 0.3

    if word in rhythm_words:
      word_rank[key] += 0.3

    split_synonyms = [s for syn in synonyms for s in syn.split()]

    if word in synonyms:
      word_rank[key] += 0.4
    if word in freq_words_for_theme:
      word_rank[key] += 0.3
    
    if word in sent and len(word)>3:
      word_rank[key] -= 10
    
    next_pair = (pair[-1], word)
    
    if next_pair not in all_pairs or word not in first_word:
      word_rank[key] -= 1000

  max_rank = max(list(word_rank.keys()), key= lambda x: word_rank[x])
  
  print(max_rank[1], word_rank[max_rank])
  return (max_rank[1], word_rank[max_rank])

In [294]:
'''
pair = ('the','sun')
sent = ['the', 'sun']
for j in range(10):
    next_word = generate_word(pair)
    pair = (pair[-1], next_word)
    sent.append(next_word)
    print(next_word)
    if pair in common_line_endings:
      break
'''

"\npair = ('the','sun')\nsent = ['the', 'sun']\nfor j in range(10):\n    next_word = generate_word(pair)\n    pair = (pair[-1], next_word)\n    sent.append(next_word)\n    print(next_word)\n    if pair in common_line_endings:\n      break\n"

In [295]:
synonym_list = list(syn for synonym in synonyms for syn in synonym.split() if len(syn)>3)
synonym_list

['political',
 'sympathies',
 'politics',
 'political',
 'relation',
 'political',
 'science',
 'authorities',
 'administration',
 'government',
 'activity',
 'regime',
 'governing',
 'governance',
 'government']

In [296]:
import random

def random_start_pair(choice):
  conditions= poem_cfd.conditions()
  pairs = [pair for pair in conditions if choice in pair and len(poem_cfd[pair].keys())>3]
  return pairs

In [297]:
def word_count(poem):
  return sum(len(line) for line in poem)

In [302]:
MAX_LENGTH = 10
NUM_LINES = 5

score = 0

choice = random.choice(freq_words_for_theme)

pairs = random_start_pair(choice)
if pairs:
  pair = random.choice(pairs)
else:
  pair = random.choice(list(poem_cfd.conditions()))


prev_pair = ('~~','~')
poem = []
sent = list(pair)
print(sent)
for i in range(NUM_LINES):
  for j in range(10):
      next_word, next_score = generate_word(pair, set(sent))
      score += next_score
      pair = (pair[-1], next_word)
      sent.append(next_word)
      #print(next_word)
      if len(sent)> 4 and pair in common_line_endings:
        if len(poem) == 0: MAX_LENGTH = len(sent)
        line = " ".join(sent)
        print(line)
        poem.append(line)
        next_word_1, next_score1 = generate_word(pair, set(sent))
        score += next_score1
        pair = (pair[-1], next_word_1)
        next_word_2, next_score2 = generate_word(pair, set(sent))
        score+= next_score2
        pair = (next_word_1, next_word_2)
        sent = [next_word_1,next_word_2]
        break
      elif j == MAX_LENGTH:
        line = " ".join(sent)
        print(line)
        poem.append(line)
        next_word_1, next_score1 = generate_word(pair, set(sent))
        score += next_score1
        pair = (pair[-1], next_word_1)
        next_word_2, next_score2 = generate_word(pair, set(sent))
        score += next_score2
        pair = (next_word_1, next_word_2)
        sent = [next_word_1,next_word_2]
      
        
print()
print()
for line in poem:
  print(line)

count = word_count(poem)

print("Score:", score/count)
  

['coming', 'he']
turned 2.9
to 3.548562392179414
look 3.4164265706282513
coming he turned to look
at 3.6436613055818357
the 3.5615083697234353
moment 3.672438652386497
i 3.47346818868558
was 3.5619565217391305
ready 3.662598142760352
to 4.242982456140351
go 3.4207098679959382
at the moment i was ready to go
home 3.4487854251012147
and 3.333834586466166
set 3.4275151181968115
the 3.3304597701149428
table 3.7479710144927534
home and set the table
and 3.2645512928849083
the 3.2357142857142858
people 3.656270783528894
are 3.393793390724799
leaving 3.7035135135135135
things 2.1166666666666667
we 2.6500000000000004
do 3.6799999999999997
and the people are leaving things we do
not 3.8476998904709747
have 3.424917719943579
to 3.4012238041913294
carry 3.6588330124042807
out 3.4396561857184835
the 3.7474789915966387
window 3.8170143346170304
not have to carry out the window
and 3.3411458333333335
watch 3.458669108669109


coming he turned to look
at the moment i was ready to go
home and set the 